In [51]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50, MobileNetV3Large, ConvNeXtBase
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.convnext import preprocess_input as convnext_preprocess



In [52]:
path = '/content/drive/MyDrive/Colab Notebooks/images'
os.listdir(path)

['apple fruit',
 'banana fruit',
 'kiwi fruit',
 'mango fruit',
 'orange fruit',
 'grapes fruit',
 'strawberry fruit',
 'cherry fruit',
 'chickoo fruit']

In [53]:
img_size = (256, 256)
batch_size = 8
epochs = 5
histories = {}

In [55]:
data = image_dataset_from_directory(path, image_size=img_size, batch_size=batch_size, shuffle=True, seed=123)
class_names = data.class_names
print(class_names)

Found 360 files belonging to 9 classes.
['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']


In [56]:
data_size = len(data)
train_size = int(0.8 * data_size)
test_ds = int(0.2 * data_size)


train_data = data.take(train_size).prefetch(tf.data.AUTOTUNE)
val_data = data.skip(train_size).prefetch(tf.data.AUTOTUNE)


In [57]:
model = models.Sequential({
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
})

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
histories = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.0609 - loss: 15.3448 - val_accuracy: 0.1250 - val_loss: 14.3524
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.1151 - loss: 14.1437 - val_accuracy: 0.1250 - val_loss: 13.3216
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.1023 - loss: 12.8473 - val_accuracy: 0.0972 - val_loss: 13.5071
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.1082 - loss: 12.9223 - val_accuracy: 0.1389 - val_loss: 12.9398
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.0781 - loss: 12.9079 - val_accuracy: 0.0972 - val_loss: 12.5901
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.0999 - loss: 12.5319 - val_accuracy: 0.1250 - val_loss: 12.5274
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.1046 - loss: 12.4942 - val_accuracy: 0.1389 - val_loss: 13.2557
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.1009 - loss: 13.1327 - val_accuracy: 0.1

In [58]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)              │ (None, 254, 254, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 125, 125, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 125, 125, 9)    │           297 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 125, 125, 128)  │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 492032)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_14 (Rescaling)        │ (None, 492032)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,501 (255.87 KB)

 Trainable params: 21,833 (85.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 43,668 (170.58 KB)

In [59]:
def build_transfer_model(base_model, preprocess_fn, model_name):
    inputs = tf.keras.Input(shape=(256, 256, 3))
    x = preprocess_fn(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(class_names), activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [60]:
## --- ResNet50 ---
resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
resnet_base.trainable = False
resnet_model = build_transfer_model(resnet_base, resnet_preprocess, "ResNet50")
histories["ResNet50"] = resnet_model.fit(train_data, validation_data=val_data, epochs=epochs)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.3946 - loss: 2.3370 - val_accuracy: 0.8056 - val_loss: 0.6406
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.8753 - loss: 0.3893 - val_accuracy: 0.8611 - val_loss: 0.6555
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 111s 3s/step - accuracy: 0.9055 - loss: 0.3421 - val_accuracy: 0.8889 - val_loss: 0.4238
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 0.9460 - loss: 0.1494 - val_accuracy: 0.9167 - val_loss: 0.3526
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 145s 3s/step - accuracy: 0.9706 - loss: 0.0912 - val_accuracy: 0.9306 - val_loss: 0.2819


TypeError: 'History' object does not support item assignment